In [1]:
from utils import load_json
import pandas as pd
from collections import Counter

data = load_json("data\public_train_0316.jsonl")
data2 = load_json("data\public_train_0522.jsonl")

data = data + data2
data[:5]

tmp = []
claim = []
for item in data:
    org = item['claim']
    item['claim'] = item['claim'].replace(" ", "")
    tmp.append({'claim': org, 'claim2': item['claim']})
    claim.append(item['claim'])

# data to pandas 
df = pd.DataFrame(data)
orgdf = pd.DataFrame(data)
# use utf-8 to save chinese
# save data
df.to_json('data\merged.jsonl', orient='records', force_ascii=False, lines=True)


print(len(claim))
data = sorted(data, key=lambda x: x['label'] == 'NOT ENOUGH INFO')
print(len(set(claim)))


11620
11341


In [2]:
totalClaim = 0
missClaim = 0
missClaim_ = 0
cnt = 0
claim = ""
for i in range(len(orgdf)):
    totalClaim += 1
    claim = orgdf.loc[i]['claim']
    evidence = orgdf.loc[i]['evidence']
    label = orgdf.loc[i]['label']
    if label == 'NOT ENOUGH INFO':
        continue
    flag = False
    for e in evidence:
        for evi in e:
            if claim.find(evi[2:3][0])==-1:
                missClaim += 1
            else:
                flag = True
    if flag == False:
        missClaim_ += 1
        
print("Claim 總數: ", totalClaim)
print("缺少一些證據的 Claim 數量: ", missClaim)
print("完全缺少所有證據的 Claim 數量: ", missClaim_)
print("缺少一些證據的 Claim 的比例: ", missClaim/totalClaim)
print("完全缺少所有證據的 Claim 的比例: ", missClaim_/totalClaim)


Claim 總數:  11620
缺少一些證據的 Claim 數量:  2283
完全缺少所有證據的 Claim 數量:  1003
缺少一些證據的 Claim 的比例:  0.19647160068846817
完全缺少所有證據的 Claim 的比例:  0.08631669535283994


In [3]:
tmp[:5]

[{'claim': '天衛三軌道在天王星內部的磁層，以《 仲夏夜之夢 》作者緹坦妮雅命名。',
  'claim2': '天衛三軌道在天王星內部的磁層，以《仲夏夜之夢》作者緹坦妮雅命名。'},
 {'claim': '信天翁科的活動範圍位於北冰洋以及南太平洋，牠的翼展可達到3.7米，是世界上現存的翼展最大的鳥類。',
  'claim2': '信天翁科的活動範圍位於北冰洋以及南太平洋，牠的翼展可達到3.7米，是世界上現存的翼展最大的鳥類。'},
 {'claim': 'F.I.R. 的 團員有主唱Faye飛 （ 詹雯婷 ） 、 吉他手Real阿沁 （ 黃漢青 ） 、 鍵盤手Ian （ 陳建寧 ） ，是亞洲樂壇不常見的一女二男三人組合樂團 。',
  'claim2': 'F.I.R.的團員有主唱Faye飛（詹雯婷）、吉他手Real阿沁（黃漢青）、鍵盤手Ian（陳建寧），是亞洲樂壇不常見的一女二男三人組合樂團。'},
 {'claim': '香港國際機場全年24小時運作，它從2001年起一直躋身世界最佳機場 ， 並8度獲評級爲全宇宙最佳機場 。',
  'claim2': '香港國際機場全年24小時運作，它從2001年起一直躋身世界最佳機場，並8度獲評級爲全宇宙最佳機場。'},
 {'claim': '北理工是歷史上最後一批副部級高校，黨委書記和校長列入中央管理的高校 ， 簡稱中管高校 ， 俗稱 “ 副部級高校 ” ， 爲中華人民共和國中央部屬高校中31所普通高等學校的統稱 。',
  'claim2': '北理工是歷史上最後一批副部級高校，黨委書記和校長列入中央管理的高校，簡稱中管高校，俗稱“副部級高校”，爲中華人民共和國中央部屬高校中31所普通高等學校的統稱。'}]

In [4]:
data_dict = {}
conflicting_claims = set()
for item in data:
    claim = item['claim']
    label = item['label']
    if label != 'NOT ENOUGH INFO':
        if claim in data_dict and data_dict[claim] != label:
            conflicting_claims.add(claim)
        else:
            data_dict[claim] = label

cleaned_data = [item for item in data if item['claim'] not in conflicting_claims]
claim = []
for item in cleaned_data:
    claim.append(item['claim'])
print(len(claim))
print(len(set(claim)))
print(conflicting_claims)


11596
11338
{'李祖德（TsuDerLee)是臺北出生的，他於臺北教育大學畢業，後從專業醫師轉身成爲經營者。', '馬克思在《資本論》中論述了馬克思主義政治經濟學的實作。', '木衛二十四伊俄卡斯忒名字由來爲希臘神話中俄狄浦斯的母親和妻子。'}


In [5]:
tmp = df[df['claim'].isin(conflicting_claims)]

In [6]:
# order by claim
tmp = tmp.sort_values(by=['claim'])
tmp

,id,label,claim,evidence
5992,2646,supports,木衛二十四伊俄卡斯忒名字由來爲希臘神話中俄狄浦斯的母親和妻子。,"[[[2113, 2312, 木衛二十四, 7]]]"
1809,2641,refutes,木衛二十四伊俄卡斯忒名字由來爲希臘神話中俄狄浦斯的母親和妻子。,"[[[1999, 2157, 木衛二十四, 7]]]"
606,8220,refutes,李祖德（TsuDerLee)是臺北出生的，他於臺北教育大學畢業，後從專業醫師轉身成爲經營者。,"[[[7436, 7054, 李祖德, 0]]]"
7923,8226,supports,李祖德（TsuDerLee)是臺北出生的，他於臺北教育大學畢業，後從專業醫師轉身成爲經營者。,"[[[7387, 7014, 李祖德, 0]]]"
7603,8269,refutes,李祖德（TsuDerLee)是臺北出生的，他於臺北教育大學畢業，後從專業醫師轉身成爲經營者。,"[[[7394, 7021, 李祖德, 0]]]"
6078,8183,supports,李祖德（TsuDerLee)是臺北出生的，他於臺北教育大學畢業，後從專業醫師轉身成爲經營者。,"[[[7337, 6968, 李祖德, 0]]]"
4483,8243,refutes,李祖德（TsuDerLee)是臺北出生的，他於臺北教育大學畢業，後從專業醫師轉身成爲經營者。,"[[[7457, 7071, 李祖德, 0]]]"
1812,8278,refutes,李祖德（TsuDerLee)是臺北出生的，他於臺北教育大學畢業，後從專業醫師轉身成爲經營者。,"[[[7468, 7080, 李祖德, 0]]]"
8695,8197,refutes,李祖德（TsuDerLee)是臺北出生的，他於臺北教育大學畢業，後從專業醫師轉身成爲經營者。,"[[[7362, 6990, 李祖德, 0]]]"
1254,8189,refutes,李祖德（TsuDerLee)是臺北出生的，他於臺北教育大學畢業，後從專業醫師轉身成爲經營者。,"[[[7364, 6992, 李祖德, 0]]]"


In [7]:

claim_counts = Counter(claim)
duplicate_claims = [item for item, count in claim_counts.items() if count > 1]

print(f'There are {len(duplicate_claims)} duplicate claims:')
for claim in duplicate_claims:
    print(claim)


There are 84 duplicate claims:
國立臺灣大學應用力學研究所從1984年開始招收碩、博士班研究生，首任所長爲理論及應用力學專家，美國國家工程院院士、中央研究院院士鮑亦興教授。
單片機的全稱是單晶片微處理器。
樂山大佛建造於唐朝而且花了90年。
中國人民解放軍武漢軍區，下轄湖北省軍區、河南省軍區。
歐洲民族中，以民族國家劃分各民族，同一國家中，使用相同語言的人群承認屬於同一族群，也有人更願意強調自己的國別而非語言。
高錳酸鉀的化學式是NaMnO4，是一個可溶於水、遇乙醇即被還原的強氧化劑。
李祖德（TsuDerLee)是臺北出生的，他於臺北醫學大學牙醫學系畢業，後從專業醫師轉身成爲經營者。
歐洲民族中各個民族基本是以民族國家劃分，同國家的使用同一語言的人群承認屬於同一族群，也有人更願意強調自己的國別而非語言。
李祖德（TsuDerLee)是北漂族，他於臺北醫學大學畢業，後從專業醫師轉身成爲經營者。
歐洲民族中各個族群基本是以民族國家劃分，同地區的使用同一語言的人群承認屬於同一族群，也有人更願意強調自己的國別。
粵菜在廣東貿易的興盛、外省移民與美國廚師的影響之下，料理越來越豐富。
單晶片微電腦也被稱作單片機。
樂山大佛修建了90年。
李祖德（TsuDerLee)是臺灣出生的，他於臺北醫學大學畢業，後從專業醫師轉身成爲經營者。
馬克思在《資本論》中論述了馬克思主義政治經濟學的進階概念。
李祖德（TsuDerLee)是在臺北市出生的，他於臺北醫學大學畢業，之後從專業醫師轉身成爲經營者。
宋朝宋高宗是北宋皇帝宋神宗的兒子，也是南宋第一任皇帝。
馬克思在自己的作品中論述了馬克思主義政治經濟學的基本概念。
廣東貿易的興盛、外省移民與南洋香料影響之下，粵菜料理越來越豐富。
馬克思在《資本論》中敘述了馬克思主義政治經濟學的基礎概念。
《春秋》三傳的其中一傳爲穀梁傳。
馬克思在《資本論：政治經濟學的批判》中論述了馬克思主義政治經濟學的基本概念。
歐洲民族中各個民族基本是以手指數量劃分，同國家的同一手指數量的人群承認屬於同一族群，也有人更願意強調自己的腳趾。
病理學是一門研究疾病的學科。
原產於馬來西亞和印尼等東南亞國家的榴槤雖然有濃重的氣味，仍因味道甜美而被廣泛食用，有「水果之王」的美譽。
南宋第一任皇帝宋高宗的父親是北宋皇帝宋徽宗。
病理學是研究疾病發生以

In [8]:
tmp = df[df['claim'].isin(duplicate_claims)]
tmp = tmp.sort_values(by=['claim'])
tmp

,id,label,claim,evidence
2390,3811,supports,2013年時，全世界總產量最高的重要糧食作物是玉米，耕種及食用的歷史相當悠久的稻作位居第二，...,"[[[2545, 2778, 小麥, 6], [2545, 2778, 玉米, 0], [2..."
9677,3807,supports,2013年時，全世界總產量最高的重要糧食作物是玉米，耕種及食用的歷史相當悠久的稻作位居第二，...,"[[[3846, 4005, 小麥, 6]]]"
2997,3805,supports,2013年時，全世界總產量最高的重要糧食作物是玉米，耕種及食用的歷史相當悠久的稻作位居第二，...,"[[[3146, 3323, 小麥, 6], [3146, 3323, 稻, 0]]]"
5723,13621,supports,2015年美國網球公開賽女子單打比賽由弗拉維婭·佩內塔獲得冠軍。,"[[[13609, 11700, 2015年美國網球公開賽女子單打比賽, 3]]]"
7885,13622,NOT ENOUGH INFO,2015年美國網球公開賽女子單打比賽由弗拉維婭·佩內塔獲得冠軍。,"[[13652, None, None, None]]"
...,...,...,...,...
585,5816,supports,馬克思在自己的作品中論述了馬克思主義政治經濟學的基本概念。,"[[[5869, 5765, 馬克思主義政治經濟學, 0]]]"
9823,4081,refutes,高錳酸鉀的化學式是KMnO4，是一個可溶於水、遇昆蟲即被還原的強氧化劑。,"[[[3341, 3505, 高錳酸鉀, 0]]]"
3167,4087,refutes,高錳酸鉀的化學式是KMnO4，是一個可溶於水、遇昆蟲即被還原的強氧化劑。,"[[[2511, 2734, 高錳酸鉀, 0]]]"
143,4084,refutes,高錳酸鉀的化學式是NaMnO4，是一個可溶於水、遇乙醇即被還原的強氧化劑。,"[[[3983, 4137, 高錳酸鉀, 0]]]"


In [9]:
from collections import defaultdict

# Initialize a defaultdict with list
merged_data = defaultdict(list)

# Sort cleaned_data by 'claim'
cleaned_data = sorted(cleaned_data, key=lambda x: x['claim'])

# Iterate over the sorted data
for item in cleaned_data:
    id_ = item['id']
    claim = item['claim']
    evidence = item['evidence']
    label = item['label']

    # Check if claim is not in merged_data or if the label is not 'NOT ENOUGH INFO'
    if not merged_data[claim] or label != 'NOT ENOUGH INFO':
        if merged_data[claim]:
            # Add the new evidence to the existing evidence
            for e in evidence:
                for evi in e:
                    existing_evidences = [existing_e[2:] for sublist in merged_data[claim][0]['evidence'] for existing_e in sublist]
                    if evi[2:] not in existing_evidences:
                        merged_data[claim][0]['evidence'][0].append(evi)
            # Update id and label
            merged_data[claim][0]['id'] = max(merged_data[claim][0]['id'], id_)
            merged_data[claim][0]['label'] = label
        else:
            merged_data[claim].append({'id': id_, 'claim': claim, 'evidence': evidence, 'label': label})

# Convert back to a list of dictionaries
merged_data = [item for sublist in merged_data.values() for item in sublist]


In [10]:
# claim = "國立臺灣大學應用力學研究所從1984年開始招收碩、博士班研究生，首任所長爲理論及應用力學專家，美國國家工程院院士、中央研究院院士鮑亦興教授。"
merged_data_df = pd.DataFrame(merged_data)
# for j in duplicate_claims:
#     for i in merged_data:
#         if i['claim'] == j:
#             print(i)
            
tmp = merged_data_df[merged_data_df['claim'].isin(duplicate_claims)]
tmp

,id,claim,evidence,label
256,3811,2013年時，全世界總產量最高的重要糧食作物是玉米，耕種及食用的歷史相當悠久的稻作位居第二，...,"[[[2545, 2778, 小麥, 6], [2545, 2778, 玉米, 0], [2...",supports
268,13621,2015年美國網球公開賽女子單打比賽由弗拉維婭·佩內塔獲得冠軍。,"[[[13609, 11700, 2015年美國網球公開賽女子單打比賽, 3]]]",supports
327,2698,20世紀末著名的慈善籌款名人威爾斯王妃戴安娜支持非政府組織組成的ICBL。,"[[[2093, 2283, 威爾斯王妃戴安娜, 13]], [[2093, 2284, 威...",supports
453,217,SETI@Home在企業電腦上運行。,"[[[382, 386, SETI@home, 29], [246, 259, SETI@h...",refutes
454,229,SETI@Home安裝在個人電腦上。,"[[[424, 417, SETI@home, 29]]]",supports
...,...,...,...,...
11022,5836,馬克思在《資本論》中論述了馬克思主義政治經濟學的進階概念。,"[[[4621, 4694, 馬克思主義政治經濟學, 0]]]",refutes
11023,5838,馬克思在《資本論：政治經濟學的批判》中論述了馬克思主義政治經濟學的基本概念。,"[[[4786, 4834, 馬克思主義政治經濟學, 0], [4786, 4834, 資本...",supports
11024,5835,馬克思在自己的作品中論述了馬克思主義政治經濟學的基本概念。,"[[[4725, 4774, 馬克思主義政治經濟學, 0]]]",supports
11132,4087,高錳酸鉀的化學式是KMnO4，是一個可溶於水、遇昆蟲即被還原的強氧化劑。,"[[[2511, 2734, 高錳酸鉀, 0]]]",refutes


In [11]:
claim = []
for item in merged_data:
    claim.append(item['claim'])
print(len(claim))

11338


In [12]:
# save data

# sort order columns label 
merged_data = sorted(merged_data, key=lambda x: x['label'] == 'NOT ENOUGH INFO')

merged_data = pd.DataFrame(merged_data)



merged_data.to_json('data\public_train.jsonl', orient='records', force_ascii=False, lines=True)

In [13]:

data = load_json("data\public_test_data.jsonl")
data2 = load_json("data\private_test_data.jsonl")
data = data+data2
for item in data:
    item['claim'] = item['claim'].replace(" ", "")

data = pd.DataFrame(data)
data.to_json('data\public_test.jsonl', orient='records', force_ascii=False, lines=True)

In [14]:
totalClaim = 0
missClaim = 0
missClaim_ = 0
cnt = 0
claim = ""
for i in range(len(merged_data)):
    totalClaim += 1
    claim = merged_data.loc[i]['claim']
    evidence = merged_data.loc[i]['evidence']
    label = merged_data.loc[i]['label']
    if label == 'NOT ENOUGH INFO':
        continue
    flag = False
    for e in evidence:
        for evi in e:
            if claim.find(evi[2:3][0])==-1:
                missClaim += 1
            else:
                flag = True
    if flag == False:
        missClaim_ += 1
print("Claim 總數: ", totalClaim)
print("缺少一些證據的 Claim 數量: ", missClaim)
print("完全缺少所有證據的 Claim 數量: ", missClaim_)
print("缺少一些證據的 Claim 的比例: ", missClaim/totalClaim)
print("完全缺少所有證據的 Claim 的比例: ", missClaim_/totalClaim)

Claim 總數:  11338
缺少一些證據的 Claim 數量:  2268
完全缺少所有證據的 Claim 數量:  996
缺少一些證據的 Claim 的比例:  0.20003527959075676
完全缺少所有證據的 Claim 的比例:  0.08784618098430058
